In [43]:
import pandas as pd

melbourne_data = pd.read_csv('melb_data.csv')
melbourne_data.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [44]:
melbourne_data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

Deleting missing values from the data

In [45]:
melbourne_data = melbourne_data.dropna(axis=0)
#Same as above but with inplace=True
#melbourne_data.dropna(axis=0, inplace=True)

In [51]:
y = melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']
X= melbourne_data[melbourne_features]

In [58]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

In [59]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=0)
model.fit(train_X, train_y)

DecisionTreeRegressor(random_state=0)

In [60]:
from sklearn.metrics import mean_absolute_error
val_predictions = model.predict(val_X)
mean_absolute_error(val_y, val_predictions)

251520.819883796

In [63]:
from sklearn.metrics import mean_absolute_error
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model_mae = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model_mae.fit(train_X, train_y)
    preds_val = model_mae.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return mae

In [64]:
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  369673
Max leaf nodes: 50  		 Mean Absolute Error:  266644
Max leaf nodes: 500  		 Mean Absolute Error:  243613
Max leaf nodes: 5000  		 Mean Absolute Error:  256227


In the below code, I tried to apply n_jobs parameter to the model. It dramatically reduces the time taken to run the model. -1 means all the jobs will be in parallel. In default, it is 1.

In [70]:
import time
from sklearn.ensemble import RandomForestRegressor
forest_model = RandomForestRegressor(random_state=1,n_jobs=-1)
start = time.time()
forest_model.fit(train_X, train_y)
predictions = forest_model.predict(val_X)
end = time.time()
print("Time taken for training: ", end - start)
print(mean_absolute_error(val_y, predictions))
# 0.9250147342681885
# 0.2753736972808838

Time taken for training:  0.2650489807128906
190414.59149026


Up to this point we used values without missing values. Now we will use values with missing values.
Still we will only use numeric values.

In [84]:
data = pd.read_csv('melb_data.csv')
y = data.Price
# Numeric predictors. We first drop the price then drop the non-numeric columns.
X = data.drop(['Price'], axis=1).select_dtypes(exclude=['object'])
# Train test split of data.
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=1, train_size=0.8, test_size=0.2)

pandas.core.frame.DataFrame

In [99]:
def score_dataset(X_train, y_train, X_val, y_val):
    numeric_model = RandomForestRegressor(n_estimators=100, random_state=1, n_jobs=-1)
    numeric_model.fit(X_train, y_train)
    preds_val = numeric_model.predict(X_val)

    return mean_absolute_error(y_val, preds_val)

In [81]:
# Get names of columns with missing values
cols_with_missing = [col for col in X.columns if X[col].isnull().any()]
# Drop columns in training and validation data
reduced_X_train = train_X.drop(cols_with_missing, axis=1)
reduced_X_val = val_X.drop(cols_with_missing, axis=1)
# Print score from above function
score_dataset(reduced_X_train, y_train, reduced_X_val, val_y)

187082.57548478153

In [89]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_val = pd.DataFrame(my_imputer.transform(X_val))

imputed_X_train.columns = X_train.columns
imputed_X_val.columns = X_val.columns

score_dataset(imputed_X_train, y_train, imputed_X_val, val_y)


<class 'pandas.core.frame.DataFrame'>


174917.69150711832

In [90]:
X_train_plus = X_train.copy()
X_val_plus = X_val.copy()

# For every column with missing values, we create another columd filled with True and False. True is where the missing value is present.
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_val_plus[col + '_was_missing'] = X_val_plus[col].isnull()

my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_val_plus = pd.DataFrame(my_imputer.transform(X_val_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_val_plus.columns = X_val_plus.columns

score_dataset(imputed_X_train_plus, y_train, imputed_X_val_plus, val_y)

177346.30320324007

From this point on, we will use categorical variables.

In [94]:
y = data.Price
X = data.drop(['Price'], axis=1)

X_train_full, X_val_full, y_train, y_val = train_test_split(X, y, random_state=1, train_size=0.8, test_size=0.2)

columns_with_missing_values = [col for col in X_train_full.columns if X_train_full[col].isnull().any()]

X_train_full.drop(columns_with_missing_values, axis=1, inplace=True)
X_val_full.drop(columns_with_missing_values, axis=1, inplace=True)


low_cardinality_columns = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]

numerical_columns = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

predictor_columns = low_cardinality_columns + numerical_columns
X_train = X_train_full[predictor_columns].copy()
X_val = X_val_full[predictor_columns].copy()

# S refers to a series of True/False values of wheter the value is categorical(True) or not(False).
s = (X_train.dtypes == 'object')
#   Returns a list of columns that are categorical. s== True
object_cols = list(s[s].index)
#object_cols = X_train.columns[s].tolist()


In [100]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_val = X_val.select_dtypes(exclude=['object'])
score_dataset(drop_X_train, y_train, drop_X_val, y_val)

176556.1092096132

In [104]:
from sklearn.preprocessing import OrdinalEncoder

label_X_train = X_train.copy()
labeled_X_val = X_val.copy()

ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
labeled_X_val[object_cols] = ordinal_encoder.transform(X_val[object_cols])

print(score_dataset(label_X_train, y_train, labeled_X_val, y_val))


165018.58378760781


In [106]:
object_cols

['Type', 'Method', 'Regionname']

In [127]:
from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_val = pd.DataFrame(OH_encoder.transform(X_val[object_cols]))
print(OH_encoder.categories_)
OH_cols_train.index = X_train.index
OH_cols_val.index = X_val.index

num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_val.drop(object_cols, axis=1)

OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_val], axis=1)

score_dataset(OH_X_train, y_train, OH_X_valid, y_val)

[array(['h', 't', 'u'], dtype=object), array(['PI', 'S', 'SA', 'SP', 'VB'], dtype=object), array(['Eastern Metropolitan', 'Eastern Victoria',
       'Northern Metropolitan', 'Northern Victoria',
       'South-Eastern Metropolitan', 'Southern Metropolitan',
       'Western Metropolitan', 'Western Victoria'], dtype=object)]


/Users/aliakkaya/PycharmProjects/PersonalArea/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/aliakkaya/PycharmProjects/PersonalArea/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


162363.08246142787

Until this point in categorical variable both validation and training sets have the same number of unique values for each categorical variable. Now we will learn how to handle if validation data has different values for the same categorical variable.

In [125]:
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

good_label_cols = [col for col in object_cols if
                   set(X_val[col]).issubset(set(X_train[col]))]
bad_label_cols = list(set(object_cols)-set(good_label_cols))

label_X_train_2 = X_train.drop(bad_label_cols, axis=1)
labeled_X_val_2 = X_val.drop(bad_label_cols, axis=1)

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount,0,...,6,7,8,9,10,11,12,13,14,15
1041,3,11.2,3186.0,3.0,1.0,366.0,-37.90380,145.00010,10579.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1989,3,7.8,3058.0,3.0,1.0,238.0,-37.75390,144.96120,11204.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10157,3,5.2,3056.0,3.0,1.0,439.0,-37.77047,144.97005,11918.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1711,2,11.4,3163.0,2.0,1.0,0.0,-37.88630,145.06600,7822.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
11565,4,11.0,3018.0,4.0,2.0,615.0,-37.87057,144.83623,5301.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,3,13.9,3165.0,3.0,1.0,686.0,-37.91130,145.07650,10969.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5192,2,11.2,3073.0,2.0,1.0,452.0,-37.72710,145.00090,21650.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
12172,3,31.7,3429.0,3.0,2.0,563.0,-37.57499,144.75329,14092.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
235,5,6.3,3143.0,8.0,2.0,693.0,-37.86380,145.02290,4836.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [134]:
#Create algorithm for Twitter


'acbb abccello'